In [61]:
import os
import pandas as pd

from dynaconf import Dynaconf

from llmcoder.utils import get_config_dir, get_data_dir

In [62]:
# Get all configs
config_dir = get_config_dir()
config_file_list = sorted(file for file in os.listdir(config_dir) if file.endswith(".yaml"))
config_list = [Dynaconf(settings_file=os.path.join(config_dir, config_name)) for config_name in config_file_list]
config_name_list = [os.path.splitext(config_name)[0] for config_name in config_file_list]

for config_name in config_name_list:
    print(config_name)

level_0__ft__mypy__5_steps__1_choice
level_0__ft__mypy_signature_5_steps__1_choice
level_0__ft__mypy_signature_5_steps__3_choices
level_0__ft__mypy_signature_gptscore_5_steps__3_choices
level_0__ft__no_analyzers__1_step
level_0__gpt-3.5__no_analyzers__1_step
level_1__ft__mypy__5_steps__1_choice
level_1__ft__mypy_signature_5_steps__1_choice
level_1__ft__mypy_signature_5_steps__3_choices
level_1__ft__mypy_signature_gptscore_5_steps__3_choices
level_1__ft__no_analyzers__1_step
level_1__gpt-3.5__no_analyzers__1_step
level_2__ft__mypy__5_steps__1_choice
level_2__ft__mypy_signature_5_steps__1_choice
level_2__ft__mypy_signature_5_steps__3_choices
level_2__ft__mypy_signature_gptscore_5_steps__3_choices
level_2__ft__no_analyzers__1_step
level_2__gpt-3.5__no_analyzers__1_step


In [63]:
# /data/name/of/dataset/eval/<config_name>/<run_id>/metrics.csv

In [64]:
# Load all metrics.csv files into a dict
metrics_dict = {}

for config_name, config in zip(config_name_list, config_list):
    dataset = config.get('dataset')
    path_to_eval = os.path.join(get_data_dir(dataset), 'eval', config_name)

    metrics_dict[config_name] = {}

    for run_id in os.listdir(path_to_eval):
        path_to_metrics = os.path.join(path_to_eval, run_id, 'metrics.csv')
        if os.path.exists(path_to_metrics):
            with open(path_to_metrics, 'r') as f:
                metrics_dict[config_name][run_id] = pd.read_csv(f, index_col=0)

In [65]:
# Dimensions: (config, run, example, metric)

In [66]:
metrics_dict['level_0__ft__mypy__5_steps__1_choice']['2024-01-13_16-46-25']

,levenshtein_distance_score,bleu_score,trf_similarity_score,sequence_matcher_score,gpt_reviewer_score,loops_required_score,tokens_used_score,agility_score,time_score
0,0,1.000000,1.000000,1.000000,0.0,0,299,0.0,0.510669
1,298,0.111787,0.813752,0.456835,0.0,0,1075,0.0,1.627634
2,255,0.296616,0.693067,0.419530,-4.0,5,3955,0.0,18.539025
3,93,0.381036,0.855461,0.686869,-2.0,0,481,0.0,0.694365


## Average across runs

In [67]:
from functools import reduce

In [68]:
# Create two new dataframes for the mean and std of each entry in the dataframe stored in metrics_dict['<config_name>']['<run_id>'] across all runs
# The index of the new dataframe should be the same as the index of the original dataframe

for k, v in metrics_dict.items():
    # mean
    metrics_dict[k]['mean'] = reduce(lambda a, b: a.add(b, fill_value=0), metrics_dict[k].values()) / len(metrics_dict[k])

    # std
    metrics_dict[k]['std'] = reduce(lambda a, b: a.add(b, fill_value=0), [((metrics_dict[k][run_id] - metrics_dict[k]['mean']) ** 2) for run_id in metrics_dict[k]]) / len(metrics_dict[k])

In [69]:
metrics_dict[k]['mean'].apply(lambda x: x.round(3))

,levenshtein_distance_score,bleu_score,trf_similarity_score,sequence_matcher_score,gpt_reviewer_score,loops_required_score,tokens_used_score,agility_score,time_score
0,537.5,0.047,0.413,0.041,0.5,0.0,562.5,0.0,5.351
1,384.0,0.084,0.230,0.087,-6.0,0.0,564.5,0.0,7.325


In [71]:
# Create a combined, formatted dataframe with each value formatted as `mean ± std`

for k, v in metrics_dict.items():
    metrics_dict[k]['formatted'] = pd.DataFrame(index=metrics_dict[k]['mean'].index, columns=metrics_dict[k]['mean'].columns)
    for col in metrics_dict[k]['mean'].columns:
        metrics_dict[k]['formatted'][col] = metrics_dict[k]['mean'][col].astype(float).map("{:.1f}".format) + ' ± ' + metrics_dict[k]['std'][col].astype(float).map("{:.1f}".format)

In [74]:
for k, v in metrics_dict.items():
    print(k)
    display(metrics_dict[k]['formatted'])
    print()

level_0__ft__mypy__5_steps__1_choice


,levenshtein_distance_score,bleu_score,trf_similarity_score,sequence_matcher_score,gpt_reviewer_score,loops_required_score,tokens_used_score,agility_score,time_score
0,0.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,299.0 ± 0.0,0.0 ± 0.0,0.5 ± 0.0
1,298.5 ± 0.2,0.1 ± 0.0,0.8 ± 0.0,0.4 ± 0.0,0.5 ± 0.2,2.5 ± 4.2,3332.0 ± 3396032.7,0.0 ± 0.0,20.7 ± 241.6
2,312.0 ± 2166.0,0.1 ± 0.0,0.4 ± 0.1,0.2 ± 0.0,-7.0 ± 6.0,3.0 ± 2.7,2701.0 ± 1048344.0,0.5 ± 0.2,14.0 ± 13.7
3,93.0 ± 0.0,0.4 ± 0.0,0.9 ± 0.0,0.7 ± 0.0,-2.0 ± 0.0,0.0 ± 0.0,481.0 ± 0.0,0.0 ± 0.0,0.7 ± 0.0



level_0__ft__mypy_signature_5_steps__1_choice


,levenshtein_distance_score,bleu_score,trf_similarity_score,sequence_matcher_score,gpt_reviewer_score,loops_required_score,tokens_used_score,agility_score,time_score
0,0.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0,299.0 ± 0.0,0.0 ± 0.0,0.5 ± 0.0
1,233.5 ± 748.2,0.3 ± 0.0,0.8 ± 0.0,0.5 ± 0.0,1.0 ± 0.0,0.0 ± 0.0,1085.5 ± 204.2,0.0 ± 0.0,1.2 ± 0.0
2,374.0 ± 0.0,0.0 ± 0.0,0.1 ± 0.0,0.0 ± 0.0,-10.0 ± 0.0,1.5 ± 0.2,1781.5 ± 68480.2,0.7 ± 0.1,11.4 ± 13.7
3,158.5 ± 2860.2,0.4 ± 0.0,0.7 ± 0.0,0.6 ± 0.0,0.5 ± 0.2,0.5 ± 0.2,722.5 ± 38881.5,0.5 ± 0.2,7.6 ± 30.9



level_0__ft__mypy_signature_5_steps__3_choices


...

,levenshtein_distance_score,bleu_score,trf_similarity_score,sequence_matcher_score,gpt_reviewer_score,loops_required_score,tokens_used_score,agility_score,time_score
0,84.0 ± 864.0,0.2 ± 0.0,0.4 ± 0.0,0.3 ± 0.0,-1.5 ± 4.2,0.0 ± 0.0,422.0 ± 112.7,0.0 ± 0.0,2.3 ± 0.0
1,426.0 ± 0.0,0.0 ± 0.0,0.4 ± 0.0,0.1 ± 0.0,-4.5 ± 0.2,0.0 ± 0.0,534.0 ± 0.0,0.0 ± 0.0,2.1 ± 0.0



level_2__ft__no_analyzers__1_step


,levenshtein_distance_score,bleu_score,trf_similarity_score,sequence_matcher_score,gpt_reviewer_score,loops_required_score,tokens_used_score,agility_score,time_score
0,85.0 ± 192.7,0.2 ± 0.0,0.5 ± 0.0,0.3 ± 0.0,-1.0 ± 2.7,0.0 ± 0.0,423.5 ± 20.2,0.0 ± 0.0,0.6 ± 0.0
1,431.5 ± 20.2,0.0 ± 0.0,0.3 ± 0.0,0.1 ± 0.0,-4.5 ± 4.2,0.0 ± 0.0,534.5 ± 0.2,0.0 ± 0.0,0.5 ± 0.0



level_2__gpt-3.5__no_analyzers__1_step


,levenshtein_distance_score,bleu_score,trf_similarity_score,sequence_matcher_score,gpt_reviewer_score,loops_required_score,tokens_used_score,agility_score,time_score
0,537.5 ± 32413.5,0.0 ± 0.0,0.4 ± 0.0,0.0 ± 0.0,0.5 ± 1.5,0.0 ± 0.0,562.5 ± 4004.2,0.0 ± 0.0,5.4 ± 7.1
1,384.0 ± 42.7,0.1 ± 0.0,0.2 ± 0.0,0.1 ± 0.0,-6.0 ± 0.0,0.0 ± 0.0,564.5 ± 28.2,0.0 ± 0.0,7.3 ± 22.5
